In [2]:
import math
import sqlite3 
import openpyxl
import os
from win32com.client import Dispatch
import openpyxl.utils as xlutils
import openpyxl.utils.cell as xlcell
from openpyxl.styles import PatternFill
from copy import copy
import openpyxl.chart

#V100

#***************************************************************************************************************************
#This Version will match every month where there is a match
#***************************************************************************************************************************

# Step 0: Prompt the user for the desired name
desired_name = input("Enter the desired name: ")

# Step 1: Connect to the SQLite database - Edward's One-Drive Path
connection = sqlite3.connect('ReturnsDB.db')
cursor = connection.cursor()

# Step 2: Execute SQL query with the user-provided name
query = f"SELECT Name, Month, Return FROM NMR WHERE Name = '{desired_name}';"
cursor.execute(query)

# Step 3: Fetch query results
query_results = cursor.fetchall()

# Check if query_results is empty
if not query_results:
    print(f"The name '{desired_name}' does not exist in the database. Halting processing.")
    cursor.close()
    connection.close()
    exit()

# Step 4: Open the original Excel workbook
original_file = 'PCMSingleComp.xlsx'
original_workbook = openpyxl.load_workbook(original_file)

# Step 5: Create a copy of the original workbook only if the name is found
base_path = os.path.dirname(os.path.abspath(original_file))
copy_prefix = f'PCMSingleComp_{desired_name}_Copy'  # Replace "copy" with the desired name

# Find a unique copy name
copy_name = copy_prefix
copy_number = 1
while os.path.exists(os.path.join(base_path, f'{copy_name}.xlsx')):
    copy_number += 1
    copy_name = f'{copy_prefix}V{copy_number:02d}'
copy_path = os.path.join(base_path, f'{copy_name}.xlsx')

# Create a copy of the original workbook using Excel COM automation
xl = Dispatch("Excel.Application")
xl.Visible = False

wb1 = xl.Workbooks.Open(Filename=os.path.abspath(original_file))
wb2 = xl.Workbooks.Add()
ws1 = wb1.Worksheets(1)
ws1.Copy(Before=wb2.Worksheets(1))
wb2.SaveAs(Filename=os.path.abspath(copy_path))

wb1.Close(SaveChanges=False)
wb2.Close(SaveChanges=True)
xl.Quit()


# Step 6: Open the copied workbook and get tge 
copied_workbook = openpyxl.load_workbook(copy_path)

# Step 7: Get the copied worksheet
copied_sheet = copied_workbook.active
row_number = 15  # Start from the second row (assuming the first row is for headers)

# Iterate through each row of query results
for row_data in query_results:
    name = row_data[0]  # Name
    month_text = row_data[1]  # Month
    returns = row_data[2]  # Return
    
    copied_sheet['C14'].value = desired_name  # Set the value of cell C14 with the Name

    print(f"Processing record: Name={name}, Month={month_text}, Return={returns}")

    # Reset the row_number for each record
    row_number = 15

    # Find the cell that matches the month and update the 'Return' column
    match_found = False  # Flag to track if a match is found

    for row in copied_sheet.iter_rows(min_row=row_number, max_row=158, min_col=1, max_col=1):
        cell = row[0]
        month = str(cell.value).strip()  # Get the month from the cell as a string, stripping any leading or trailing spaces

        if month.lower() == month_text.lower():  # Case-insensitive comparison
            print("Match found!")
            copied_sheet.cell(row=row_number, column=3).value = returns  # Paste returns in the third column (C column)
            match_found = True  # Set the flag to True
        row_number += 1
    if not match_found:
        print("No match found.")

# Step 8: Delete rows without entry in C column
rows_to_delete = []

for row in copied_sheet.iter_rows(min_row=15, max_row=155, min_col=3, max_col=3):
    cell = row[0]
    if cell.value is None:
        rows_to_delete.append(cell.row)

for row_index in reversed(rows_to_delete):
    copied_sheet.delete_rows(row_index)

print("Finished iterating over the worksheet.")


# Step 9: Drag down the formulas in columns Q, R, T, and U together


    # Get the starting row (row 15) and the last row with data (based on column C)
starting_row = 15
last_row = copied_sheet.max_row

    # Increment the formulas for each column and paste the results
for row in range(starting_row, last_row + 1):
    # Increment the formula for column Q
    formula_q = f"=(B{row}*Q{row-1})+Q{row-1}"
    copied_sheet[f"Q{row}"].value = formula_q

    # Increment the formula for column R
    formula_r = f"=MIN((Q{row}-MAX($Q$14:Q{row}))/MAX($Q$14:Q{row}),0)"
    copied_sheet[f"R{row}"].value = formula_r

    # Increment the formula for column T
    formula_t = f"=(C{row}*T{row-1})+T{row-1}"
    copied_sheet[f"T{row}"].value = formula_t

    # Increment the formula for column U
    formula_u = f"=MIN((T{row}-MAX($T$14:T{row}))/MAX($T$14:T{row}),0)"
    copied_sheet[f"U{row}"].value = formula_u

print("Formulas incremented and pasted successfully.")


# Step 12: Save and Close the workbook and database connection
copied_workbook.save(copy_path)
original_workbook.close()
copied_workbook.close()
cursor.close()
connection.close()


print("Processing completed.")


Enter the desired name: Matt Rader
Processing record: Name=Matt Rader, Month=Apr-10, Return=0.005
No match found.
Processing record: Name=Matt Rader, Month=Apr-11, Return=-0.0052
No match found.
Processing record: Name=Matt Rader, Month=Apr-13, Return=0.0252
Match found!
Processing record: Name=Matt Rader, Month=Apr-14, Return=0.0124
Match found!
Processing record: Name=Matt Rader, Month=Apr-15, Return=0.0054
Match found!
Processing record: Name=Matt Rader, Month=Apr-16, Return=0.0148
Match found!
Processing record: Name=Matt Rader, Month=Apr-17, Return=0.0068
Match found!
Processing record: Name=Matt Rader, Month=Apr-18, Return=0.0046
Match found!
Processing record: Name=Matt Rader, Month=Aug-10, Return=0.0313
No match found.
Processing record: Name=Matt Rader, Month=Aug-11, Return=0.0058
Match found!
Processing record: Name=Matt Rader, Month=Aug-12, Return=0.0049
Match found!
Processing record: Name=Matt Rader, Month=Aug-13, Return=0.0005
Match found!
Processing record: Name=Matt Rad

In [1]:
import math
import sqlite3 
import openpyxl
import os
from win32com.client import Dispatch
import openpyxl.utils as xlutils
import openpyxl.utils.cell as xlcell
from openpyxl.styles import PatternFill
from copy import copy
import openpyxl.chart

#V200

#***************************************************************************************************************************
#This Version will only keep the longest streak of matched months 
#***************************************************************************************************************************

# Step 0: Prompt the user for the desired name
desired_name = input("Enter the desired name: ")

# Step 1: Connect to the SQLite database - Edward's One-Drive Path
connection = sqlite3.connect('ReturnsDB.db')
cursor = connection.cursor()

# Step 2: Execute SQL query with the user-provided name
query = f"SELECT Name, Month, Return FROM NMR WHERE Name = '{desired_name}';"
cursor.execute(query)

# Step 3: Fetch query results
query_results = cursor.fetchall()

# Check if query_results is empty
if not query_results:
    print(f"The name '{desired_name}' does not exist in the database. Halting processing.")
    cursor.close()
    connection.close()
    exit()

# Step 4: Open the original Excel workbook
original_file = 'PCMSingleComp.xlsx'
original_workbook = openpyxl.load_workbook(original_file)

# Step 5: Create a copy of the original workbook only if the name is found
base_path = os.path.dirname(os.path.abspath(original_file))
copy_prefix = f'PCMSingleComp_{desired_name}_Copy'  # Replace "copy" with the desired name

# Find a unique copy name
copy_name = copy_prefix
copy_number = 1
while os.path.exists(os.path.join(base_path, f'{copy_name}.xlsx')):
    copy_number += 1
    copy_name = f'{copy_prefix}V{copy_number:02d}'
copy_path = os.path.join(base_path, f'{copy_name}.xlsx')

# Create a copy of the original workbook using Excel COM automation
xl = Dispatch("Excel.Application")
xl.Visible = False

wb1 = xl.Workbooks.Open(Filename=os.path.abspath(original_file))
wb2 = xl.Workbooks.Add()
ws1 = wb1.Worksheets(1)
ws1.Copy(Before=wb2.Worksheets(1))
wb2.SaveAs(Filename=os.path.abspath(copy_path))

wb1.Close(SaveChanges=False)
wb2.Close(SaveChanges=True)
xl.Quit()


# Step 6: Open the copied workbook and get the copied worksheet
copied_workbook = openpyxl.load_workbook(copy_path)
copied_sheet = copied_workbook.active

# Step 7: Keep track of the longest streak of consecutive matches and paste query results into Excel worksheet
current_streak = 0
longest_streak = 0

copied_sheet['C14'].value = desired_name  # Set the value of cell C14 with the Name

for row in copied_sheet.iter_rows(min_row=15, max_row=158, min_col=1, max_col=1):
    cell = row[0]
    month = str(cell.value).strip()  # Get the month from the cell as a string, stripping any leading or trailing spaces

    # Reset the streak counters and query results pointer for each cell
    cursor.execute(query)
    query_results = cursor.fetchall()

    match_found = False  # Flag to track if a match is found

    for row_data in query_results:
        name = row_data[0]  # Name
        month_text = row_data[1]  # Month
        returns = row_data[2]  # Return

        if month.lower() == month_text.lower():  # Case-insensitive comparison
            print(f"Match found for cell {cell.coordinate}! Processing record: Name={name}, Month={month_text}, Return={returns}")
            copied_sheet.cell(row=cell.row, column=3).value = returns  # Paste returns in the third column (C column)
            match_found = True  # Set the flag to True
            break  # Exit the inner loop once a match is found

    # Update the streak counters and the longest streak
    if match_found:
        current_streak += 1
        print(current_streak)
        if current_streak > longest_streak:
            longest_streak = current_streak
    else:
        current_streak = 0

    # Move the longest_streak update here, after processing each cell
    if current_streak > longest_streak:
        longest_streak = current_streak

    print(f"Current streak: {current_streak}, Longest streak: {longest_streak}")

        
# Step 8: Find the longest streak of consecutive matches
current_streak = 0
longest_streak = 0
start_index = None
end_index = None

for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=1)
    month = str(cell.value).strip()  # Get the month from the cell as a string, stripping any leading or trailing spaces

    # Reset the streak counters and query results pointer for each cell
    cursor.execute(query)
    query_results = cursor.fetchall()

    match_found = False  # Flag to track if a match is found

    for row_data in query_results:
        name = row_data[0]  # Name
        month_text = row_data[1]  # Month
        returns = row_data[2]  # Return

        if month.lower() == month_text.lower():  # Case-insensitive comparison
            copied_sheet.cell(row=row_index, column=3).value = returns  # Paste returns in the third column (C column)
            match_found = True  # Set the flag to True
            break  # Exit the inner loop once a match is found

    if match_found:
        current_streak += 1
        if current_streak > longest_streak:
            longest_streak = current_streak
            start_index = row_index - longest_streak + 1
            end_index = row_index
    else:
        current_streak = 0

print(f"Longest streak: {longest_streak}, Start index: {start_index}, End index: {end_index}")

# Step 9: Mark rows that are not part of the longest streak for deletion
for row_index in range(15, copied_sheet.max_row + 1):
    if row_index < start_index or row_index > end_index:
        copied_sheet.cell(row=row_index, column=4).value = "DELETE"

print("Marked rows for deletion.")

# Step 10: Delete marked rows
rows_to_delete = []

for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=4)
    if cell.value == "DELETE":
        rows_to_delete.append(row_index)

for row_index in reversed(rows_to_delete):
    copied_sheet.delete_rows(row_index)

print("Finished deleting rows.")
# Step 11: Drag down the formulas in columns Q, R, T, and U together


    # Get the starting row (row 15) and the last row with data (based on column C)
starting_row = 15
last_row = copied_sheet.max_row

    # Increment the formulas for each column and paste the results
for row in range(starting_row, last_row + 1):
    # Increment the formula for column Q
    formula_q = f"=(B{row}*Q{row-1})+Q{row-1}"
    copied_sheet[f"Q{row}"].value = formula_q

    # Increment the formula for column R
    formula_r = f"=MIN((Q{row}-MAX($Q$14:Q{row}))/MAX($Q$14:Q{row}),0)"
    copied_sheet[f"R{row}"].value = formula_r

    # Increment the formula for column T
    formula_t = f"=(C{row}*T{row-1})+T{row-1}"
    copied_sheet[f"T{row}"].value = formula_t

    # Increment the formula for column U
    formula_u = f"=MIN((T{row}-MAX($T$14:T{row}))/MAX($T$14:T{row}),0)"
    copied_sheet[f"U{row}"].value = formula_u

print("Formulas incremented and pasted successfully.")


# Step 12: Save and Close the workbook and database connection
copied_workbook.save(copy_path)
original_workbook.close()
copied_workbook.close()
cursor.close()
connection.close()

print("Processing completed.")

Enter the desired name: Clark Hsu
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Current streak: 0, Longest streak: 0
Curr

Longest streak: 102, Start index: 57, End index: 158
Marked rows for deletion.
Finished deleting rows.
Formulas incremented and pasted successfully.
Processing completed.


In [1]:
import sqlite3
import os
from win32com.client import Dispatch
import openpyxl

#V300

#***************************************************************************************************************************
#This Version will only keep a selected RANGE
#***************************************************************************************************************************

# Prompt the user for the desired name
desired_name = input("Enter the desired name: ")

# Prompt the user for the starting and ending dates in mmm-yy format
start_date_str = input("Enter the starting date (mmm-yy): ")
end_date_str = input("Enter the ending date (mmm-yy): ")

# Connect to the SQLite database
connection = sqlite3.connect('ReturnsDB.db')
cursor = connection.cursor()

# Execute SQL query with the user-provided name
query = f"SELECT Name, Month, Return FROM NMR WHERE Name = '{desired_name}';"
cursor.execute(query)
query_results = cursor.fetchall()

if not query_results:
    print(f"The name '{desired_name}' does not exist in the database. Halting processing.")
    cursor.close()
    connection.close()
    exit()

# Open the original Excel workbook
original_file = 'PCMSingleComp.xlsx'
original_workbook = openpyxl.load_workbook(original_file)

# Create a copy of the original workbook
base_path = os.path.dirname(os.path.abspath(original_file))
copy_prefix = f'PCMSingleComp_{desired_name}_Copy'

copy_name = copy_prefix
copy_number = 1
while os.path.exists(os.path.join(base_path, f'{copy_name}.xlsx')):
    copy_number += 1
    copy_name = f'{copy_prefix}V{copy_number:02d}'
copy_path = os.path.join(base_path, f'{copy_name}.xlsx')

xl = Dispatch("Excel.Application")
xl.Visible = False

wb1 = xl.Workbooks.Open(Filename=os.path.abspath(original_file))
wb2 = xl.Workbooks.Add()
ws1 = wb1.Worksheets(1)
ws1.Copy(Before=wb2.Worksheets(1))
wb2.SaveAs(Filename=os.path.abspath(copy_path))

wb1.Close(SaveChanges=False)
wb2.Close(SaveChanges=True)
xl.Quit()

copied_workbook = openpyxl.load_workbook(copy_path)
copied_sheet = copied_workbook.active

# Find the starting and ending rows based on the selected starting month
starting_row = None
for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=1)
    month = str(cell.value).strip()

    if month.lower() == start_date_str.lower():
        starting_row = row_index
        print(f"Starting row is: '{starting_row}'")
        break

if starting_row is None:
    print(f"Starting month '{start_date_str}' not found in the Excel sheet. Halting processing.")
    cursor.close()
    connection.close()
    exit()
    
ending_row = None
for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=1)
    month = str(cell.value).strip()

    if month.lower() == end_date_str.lower():
        ending_row = row_index
        print(f"Ending row is: '{ending_row}'")
        break

if ending_row is None:
    print(f"Ending month '{end_date_str}' not found in the Excel sheet. Halting processing.")
    cursor.close()
    connection.close()
    exit()

copied_sheet['C14'].value = desired_name  # Set the value of cell C14 with the Name
    
# Process query results and paste data in Excel sheet
current_streak = 0
longest_streak = 0

for row_index in range(starting_row, ending_row + 1):
    cell = copied_sheet.cell(row=row_index, column=1)
    month = str(cell.value).strip()

    match_found = False
    for row_data in query_results:
        name = row_data[0]
        month_text = row_data[1]
        returns = row_data[2]

        if month.lower() == month_text.lower():
            copied_sheet.cell(row=row_index, column=3).value = returns
            match_found = True
            break
                       
    if match_found:
        current_streak += 1
        if current_streak > longest_streak:
            longest_streak = current_streak
    else:
        current_streak = 0

# Mark rows that are not part of the longest streak for deletion
for row_index in range(15, copied_sheet.max_row + 1):
    if row_index < starting_row or row_index > starting_row + longest_streak - 1:
        copied_sheet.cell(row=row_index, column=4).value = "DELETE"

print("Marked rows for deletion.")

# Delete marked rows
rows_to_delete = []

for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=4)
    if cell.value == "DELETE":
        rows_to_delete.append(row_index)

for row_index in reversed(rows_to_delete):
    copied_sheet.delete_rows(row_index)

print("Finished deleting rows.")

# Drag down the formulas in columns Q, R, T, and U together
starting_row = 15
last_row = copied_sheet.max_row

for row in range(starting_row, last_row + 1):
    formula_q = f"=(B{row}*Q{row-1})+Q{row-1}"
    copied_sheet[f"Q{row}"].value = formula_q

    formula_r = f"=MIN((Q{row}-MAX($Q$14:Q{row}))/MAX($Q$14:Q{row}),0)"
    copied_sheet[f"R{row}"].value = formula_r

    formula_t = f"=(C{row}*T{row-1})+T{row-1}"
    copied_sheet[f"T{row}"].value = formula_t

    formula_u = f"=MIN((T{row}-MAX($T$14:T{row}))/MAX($T$14:T{row}),0)"
    copied_sheet[f"U{row}"].value = formula_u

print("Formulas incremented and pasted successfully.")

# Save and Close the workbook and database connection
copied_workbook.save(copy_path)
original_workbook.close()
copied_workbook.close()
cursor.close()
connection.close()

print("Processing completed.")


Enter the desired name: GeorgeAzarias
Enter the starting date (mmm-yy): May-18
Enter the ending date (mmm-yy): Dec-21
The name 'GeorgeAzarias' does not exist in the database. Halting processing.
Starting row is: '97'
Ending row is: '140'
Marked rows for deletion.
Finished deleting rows.
Formulas incremented and pasted successfully.


ProgrammingError: Cannot operate on a closed database.

In [ ]:
import sqlite3
import os
from win32com.client import Dispatch
import openpyxl

#V305

#***************************************************************************************************************************
#This Version will only keep a selected CONSECUTIVE RANGE
#***************************************************************************************************************************

# Prompt the user for the desired name
desired_name = input("Enter the desired name: ")

# Prompt the user for the starting and ending dates in mmm-yy format
start_date_str = input("Enter the starting date (mmm-yy): ")
end_date_str = input("Enter the ending date (mmm-yy): ")

# Connect to the SQLite database
connection = sqlite3.connect('ReturnsDB.db')
cursor = connection.cursor()

# Execute SQL query with the user-provided name
query = f"SELECT Name, Month, Return FROM NMR WHERE Name = '{desired_name}';"
cursor.execute(query)
query_results = cursor.fetchall()

if not query_results:
    print(f"The name '{desired_name}' does not exist in the database. Halting processing.")
    cursor.close()
    connection.close()
    exit()

# Open the original Excel workbook
original_file = 'PCMSingleComp.xlsx'
original_workbook = openpyxl.load_workbook(original_file)

# Create a copy of the original workbook
base_path = os.path.dirname(os.path.abspath(original_file))
copy_prefix = f'PCMSingleComp_{desired_name}_Copy'

copy_name = copy_prefix
copy_number = 1
while os.path.exists(os.path.join(base_path, f'{copy_name}.xlsx')):
    copy_number += 1
    copy_name = f'{copy_prefix}V{copy_number:02d}'
copy_path = os.path.join(base_path, f'{copy_name}.xlsx')

xl = Dispatch("Excel.Application")
xl.Visible = False

wb1 = xl.Workbooks.Open(Filename=os.path.abspath(original_file))
wb2 = xl.Workbooks.Add()
ws1 = wb1.Worksheets(1)
ws1.Copy(Before=wb2.Worksheets(1))
wb2.SaveAs(Filename=os.path.abspath(copy_path))

wb1.Close(SaveChanges=False)
wb2.Close(SaveChanges=True)
xl.Quit()

copied_workbook = openpyxl.load_workbook(copy_path)
copied_sheet = copied_workbook.active

# Find the starting and ending rows based on the selected starting month
starting_row = None
for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=1)
    month = str(cell.value).strip()

    if month.lower() == start_date_str.lower():
        starting_row = row_index
        print(f"Starting row is: '{starting_row}'")
        break

if starting_row is None:
    print(f"Starting month '{start_date_str}' not found in the Excel sheet. Halting processing.")
    cursor.close()
    connection.close()
    exit()
    
ending_row = None
for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=1)
    month = str(cell.value).strip()

    if month.lower() == end_date_str.lower():
        ending_row = row_index
        print(f"Ending row is: '{ending_row}'")
        break

if ending_row is None:
    print(f"Ending month '{end_date_str}' not found in the Excel sheet. Halting processing.")
    cursor.close()
    connection.close()
    exit()

copied_sheet['C14'].value = desired_name  # Set the value of cell C14 with the Name
    
# Process query results and paste data in Excel sheet

for row_index in range(starting_row, ending_row + 1):
    cell = copied_sheet.cell(row=row_index, column=1)
    month = str(cell.value).strip()

    match_found = False
    for row_data in query_results:
        name = row_data[0]
        month_text = row_data[1]
        returns = row_data[2]

        if month.lower() == month_text.lower():
            copied_sheet.cell(row=row_index, column=3).value = returns
            match_found = True
            break        
                       
    if not match_found:
        print(f"Range is not consecutive. No match found for {month}")
        print("Halting processing.")
        copied_sheet.cell(row=row_index, column=1).value = month_text
        cursor.close()
        connection.close()
        sys.exit()  # Halt the entire script here
        
        
# Mark rows that are not part of the longest streak for deletion
for row_index in range(15, copied_sheet.max_row + 1):
    if row_index < starting_row or row_index > starting_row + longest_streak - 1:
        copied_sheet.cell(row=row_index, column=4).value = "DELETE"

print("Marked rows for deletion.")

# Delete marked rows
rows_to_delete = []

for row_index in range(15, copied_sheet.max_row + 1):
    cell = copied_sheet.cell(row=row_index, column=4)
    if cell.value == "DELETE":
        rows_to_delete.append(row_index)

for row_index in reversed(rows_to_delete):
    copied_sheet.delete_rows(row_index)

print("Finished deleting rows.")

# Drag down the formulas in columns Q, R, T, and U together
starting_row = 15
last_row = copied_sheet.max_row

for row in range(starting_row, last_row + 1):
    formula_q = f"=(B{row}*Q{row-1})+Q{row-1}"
    copied_sheet[f"Q{row}"].value = formula_q

    formula_r = f"=MIN((Q{row}-MAX($Q$14:Q{row}))/MAX($Q$14:Q{row}),0)"
    copied_sheet[f"R{row}"].value = formula_r

    formula_t = f"=(C{row}*T{row-1})+T{row-1}"
    copied_sheet[f"T{row}"].value = formula_t

    formula_u = f"=MIN((T{row}-MAX($T$14:T{row}))/MAX($T$14:T{row}),0)"
    copied_sheet[f"U{row}"].value = formula_u

print("Formulas incremented and pasted successfully.")

# Save and Close the workbook and database connection
copied_workbook.save(copy_path)
original_workbook.close()
copied_workbook.close()
cursor.close()
connection.close()

print("Processing completed.")
